Reference: 
1. SpinalNet: Deep Neural Network with Gradual Input
2. https://github.com/dipuk0506/SpinalNet


# 1.0 Read train image 

In [ ]:
import torchvision
from torchvision import transforms
import torch.utils.data as data
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

batch_size_train = 64

TRANSFORM_IMG = transforms.Compose([transforms.Resize((240, 320)), transforms.ToTensor()])

# load images in batches
TRAIN_DATA_PATH = r"C:\Users\Ho\Desktop\VIP\regression\train"
train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM_IMG)
train_data_loader = data.DataLoader(train_data, batch_size=batch_size_train, shuffle=True)

# 2.0 Train Model

In [ ]:
# regression
# first_HL = 50
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary

torch.backends.cudnn.enabled = False

first_HL = 50

class Net_R(nn.Module):

  def __init__(self):
      super(Net_R, self).__init__()
      # kernal 5, maxpooling = 2
      self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
      self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
      self.conv2_drop = nn.Dropout2d()
      # features shape = 20*57*77
      self.lru = nn.LeakyReLU()
      self.fc1 = nn.Linear(int((20*57*77)/2), first_HL)  
      self.fc2 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL) 
      self.fc3 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fc4 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fc5 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fc6 = nn.Linear(int((20*57*77)/2) + first_HL, first_HL)  
      self.fcx = nn.Linear(first_HL*6, 1)  
        

  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
    x = x.view(-1, 20*57*77)
    x1 = x[:, 0:int((20*57*77)/2)]
    
    x1 = self.lru(self.fc1(x1))
    x2 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x1], dim=1)
    x2 = self.lru(self.fc2(x2))
    x3 = torch.cat([ x[:,0:int((20*57*77)/2)], x2], dim=1)
    x3 = self.lru(self.fc3(x3))
    x4 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x3], dim=1)
    x4 = self.lru(self.fc4(x4))
    x5 = torch.cat([ x[:,0:int((20*57*77)/2)], x4], dim=1)
    x5 = self.lru(self.fc3(x5))
    x6 = torch.cat([ x[:,int((20*57*77)/2):20*57*77], x5], dim=1)
    x6 = self.lru(self.fc4(x6))

    # before fc2  
    x = torch.cat([x1, x2], dim=1)
    x = torch.cat([x, x3], dim=1)
    x = torch.cat([x, x4], dim=1)
    x = torch.cat([x, x5], dim=1)
    x = torch.cat([x, x6], dim=1)

    # fc2
    x = self.fcx(x)

    return x


# model= Net()
# model.cuda()
# summary(model, (3, 240, 320))

In [ ]:
# model4: proposed setting in Zhou, Gao, and Wu (2017)
# training

from math import sqrt
import pandas as pd
import shutil

model4 = Net_R()
optimizer = torch.optim.Adam(model4.parameters(), lr=0.001)
# lr decrease from 10^-3 to 10^-5
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=len(train_data_loader)/3, gamma=0.1)
loss_func = torch.nn.MSELoss()
EPOCH = 100

result = pd.DataFrame(columns = ['epoch', 'train_loss'])

for i in range(1, EPOCH+1):
    for batch_idx, (data, target) in enumerate(train_data_loader):
        output = model4(data)
        loss = loss_func(output, target.type(torch.FloatTensor))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(batch_idx)
        print(loss.item())
        
    joblib_file = 'model4_epoch_' + str(i) + '.pt'
    torch.save(model4, joblib_file)
    shutil.move('C:/Users/Ho/Desktop/VIP/' + joblib_file, 'C:/Users/Ho/Desktop/VIP/regression/')

    result.loc[len(result)] = [i, sqrt(loss.item())]
    filename = 'model4_result_' + str(i) + '.csv'
    result.to_csv(filename, encoding='utf-8', index=False)

0
911.1561889648438
1
579.799560546875
2
292.6016845703125
3
686.0718994140625
4
363.20672607421875
5
292.4521484375
6
337.81402587890625
7
309.3108825683594
8
525.39208984375
9
374.9990539550781
10
349.3424987792969
11
251.9168243408203
12
273.4165954589844
13
263.4012145996094
14
392.01708984375
15
210.49562072753906
16
256.4412536621094
17
281.03961181640625
18
310.64569091796875
19
241.23081970214844
20
277.9570007324219
21
264.01214599609375
22
246.24539184570312
23
261.69970703125
24
246.959228515625
25
295.2763366699219
26
213.35935974121094
27
259.2869873046875
28
334.21185302734375
29
227.46961975097656
30
233.99703979492188
31
233.9082794189453
32
218.727783203125
33
228.65579223632812
34
204.82879638671875
35
280.9809875488281
36
231.8233184814453
37
253.16970825195312
38
272.9737548828125
39
173.1568603515625
40
238.67494201660156
41
193.07357788085938
42
207.42373657226562
43
211.30567932128906
44
220.16224670410156
45
225.32101440429688
46
221.48939514160156
47
246.162185

368
152.4854278564453
0
154.213134765625
1
146.06788635253906
2
126.42294311523438
3
180.07533264160156
4
118.88109588623047
5
165.81422424316406
6
119.20384979248047
7
171.23194885253906
8
96.97061157226562
9
126.18679809570312
10
145.43994140625
11
125.98802185058594
12
166.8378448486328
13
152.48904418945312
14
169.73573303222656
15
173.2996826171875
16
173.87582397460938
17
165.91920471191406
18
148.46978759765625
19
164.41867065429688
20
138.69482421875
21
179.73275756835938
22
175.3197021484375
23
167.45896911621094
24
118.20674133300781
25
104.35517883300781
26
111.49474334716797
27
178.7615966796875
28
158.99696350097656
29
142.86663818359375
30
137.93206787109375
31
118.05477142333984
32
163.20252990722656
33
109.18824768066406
34
141.65863037109375
35
121.61409759521484
36
138.3723907470703
37
156.42333984375
38
144.3802947998047
39
154.84532165527344
40
177.20445251464844
41
182.69879150390625
42
146.06637573242188
43
131.1237335205078
44
110.65576171875
45
131.6665649414062

368
139.17564392089844
0
133.9276580810547
1
154.85879516601562
2
145.7745819091797
3
167.80770874023438
4
154.16697692871094
5
207.3783416748047
6
163.11026000976562
7
115.31271362304688
8
148.2631072998047
9
156.49502563476562
10
160.80970764160156
11
135.23443603515625
12
124.1238784790039
13
164.88133239746094
14
110.90279388427734
15
148.95852661132812
16
117.66690063476562
17
178.5122528076172
18
153.8019561767578
19
173.80990600585938
20
161.35321044921875
21
127.63220977783203
22
167.73951721191406
23
137.2127227783203
24
126.03878784179688
25
139.83155822753906
26
153.45562744140625
27
133.4658660888672
28
123.86871337890625
29
105.92424011230469
30
123.9052963256836
31
133.26290893554688
32
118.43728637695312
33
174.4036865234375
34
187.04190063476562
35
174.39723205566406
36
133.8611602783203
37
145.1649169921875
38
130.20846557617188
39
148.4287109375
40
153.0342559814453
41
154.36178588867188
42
136.8716278076172
43
117.40568542480469
44
126.00961303710938
45
142.059143066

367
160.66354370117188
368
122.35755157470703
0
153.34060668945312
1
133.0799560546875
2
153.30218505859375
3
119.03497314453125
4
146.05648803710938
5
167.36874389648438
6
125.68492889404297
7
151.18370056152344
8
125.84623718261719
9
153.16226196289062
10
103.11724853515625
11
125.438232421875
12
161.0763397216797
13
131.81600952148438
14
106.10354614257812
15
142.4085693359375
16
117.32123565673828
17
127.56759643554688
18
125.9931869506836
19
180.7394256591797
20
155.84603881835938
21
164.1256103515625
22
142.43743896484375
23
139.32240295410156
24
148.4685821533203
25
130.74986267089844
26
113.89413452148438
27
162.4954833984375
28
107.97763061523438
29
119.7046890258789
30
166.58856201171875
31
146.54698181152344
32
136.8806610107422
33
148.16021728515625
34
162.91836547851562
35
143.99151611328125
36
145.55601501464844
37
105.78857421875
38
128.2078399658203
39
130.67398071289062
40
147.9660186767578
41
146.18988037109375
42
127.39479064941406
43
149.3925018310547
44
106.0940399

367
162.81907653808594
368
130.2720184326172
0
114.56652069091797
1
146.1492156982422
2
133.70074462890625
3
114.56192779541016
4
151.16085815429688
5
175.13087463378906
6
104.30680847167969
7
124.17768859863281
8
142.82559204101562
9
132.5913848876953
10
145.4315185546875
11
95.88163757324219
12
122.3536148071289
13
130.3452606201172
14
146.91896057128906
15
155.7532196044922
16
139.28472900390625
17
174.1167755126953
18
128.63279724121094
19
144.28541564941406
20
163.4390106201172
21
122.5002212524414
22
107.28163146972656
23
130.5500030517578
24
142.5955810546875
25
138.976806640625
26
131.7798614501953
27
180.8159637451172
28
135.31796264648438
29
172.98622131347656
30
137.89968872070312
31
126.73540496826172
32
210.30189514160156
33
144.42742919921875
34
113.53994750976562
35
135.36575317382812
36
123.93661499023438
37
127.29280853271484
38
126.9755630493164
39
110.05126190185547
40
109.50331115722656
41
129.20574951171875
42
173.066162109375
43
128.96058654785156
44
96.9388427734

...

328
161.99530029296875
329
130.86074829101562
330
127.77716064453125
331
151.32122802734375
332
125.37712860107422
333
135.74478149414062
334
147.3563690185547
335
137.91026306152344
336
138.19200134277344
337
100.33865356445312
338
116.03459930419922
339
177.58876037597656
340
142.6197509765625
341
139.78097534179688
342
145.34596252441406
343
164.65472412109375
344
147.3754119873047
345
162.47186279296875
346
112.01028442382812
347
129.70065307617188
348
139.6582489013672
349
151.2652130126953
350
144.159912109375
351
182.0131072998047
352
133.9749755859375
353
143.38075256347656
354
131.88377380371094
355
103.33149719238281
356
122.63041687011719
357
80.75496673583984
358
170.21437072753906
359
135.5373077392578
360
147.69252014160156
361
108.79886627197266
362
143.1702423095703
363
129.84661865234375
364
146.42266845703125
365
136.20748901367188
366
132.82781982421875
367
115.47406005859375
368
174.734375
0
165.92807006835938
1
137.3212890625
2
193.36537170410156
3
148.547760009765

328
147.3726348876953
329
95.04991149902344
330
97.47482299804688
331
127.42889404296875
332
107.7030258178711
333
110.19955444335938
334
138.4707489013672
335
121.49150848388672
336
121.24905395507812
337
170.58767700195312
338
139.0616912841797
339
122.59872436523438
340
141.26657104492188
341
117.49832153320312
342
128.26123046875
343
176.86688232421875
344
151.33758544921875
345
154.86288452148438
346
150.12515258789062
347
159.43341064453125
348
133.77825927734375
349
106.24536895751953
350
133.46694946289062
351
133.10064697265625
352
137.58843994140625
353
145.37808227539062
354
137.3584747314453
355
112.91925048828125
356
139.3397979736328
357
153.1139678955078
358
144.54165649414062
359
129.4228057861328
360
148.5410919189453
361
126.03482818603516
362
170.509033203125
363
128.7914581298828
364
143.5529022216797
365
139.21397399902344
366
173.84771728515625
367
121.63694763183594
368
128.9438934326172
0
111.10896301269531
1
180.57833862304688
2
140.9336700439453
3
131.92037963

326
152.64724731445312
327
159.52638244628906
328
123.20272827148438
329
148.20321655273438
330
117.9717788696289
331
124.18817138671875
332
129.2982635498047
333
126.08673095703125
334
132.53640747070312
335
114.40131378173828
336
138.25189208984375
337
141.36163330078125
338
175.3639373779297
339
133.5074920654297
340
105.02952575683594
341
122.11381530761719
342
114.21957397460938
343
137.2017059326172
344
139.21795654296875
345
144.36248779296875
346
108.2594223022461
347
133.96530151367188
348
128.58885192871094
349
152.4141387939453
350
154.17880249023438
351
107.65343475341797
352
143.24374389648438
353
144.002685546875
354
145.85304260253906
355
119.57491302490234
356
169.4420623779297
357
152.39488220214844
358
151.90341186523438
359
122.90373229980469
360
95.94772338867188
361
164.08668518066406
362
135.03492736816406
363
132.64100646972656
364
130.10780334472656
365
106.71539306640625
366
145.78445434570312
367
129.61866760253906
368
145.82513427734375
0
124.54071044921875
1

325
139.19580078125
326
122.6891860961914
327
109.37210845947266
328
133.58599853515625
329
134.21969604492188
330
137.76095581054688
331
141.49273681640625
332
124.67060852050781
333
159.79795837402344
334
142.30276489257812
335
175.0005645751953
336
153.81752014160156
337
153.57275390625
338
156.60299682617188
339
121.89510345458984
340
90.63365936279297
341
166.9130096435547
342
140.31214904785156
343
118.43931579589844
344
143.66464233398438
345
160.33175659179688
346
106.15386962890625
347
127.12567901611328
348
163.6011962890625
349
146.88912963867188
350
122.79254150390625
351
110.10826873779297
352
119.70238494873047
353
163.02032470703125
354
135.80853271484375
355
164.68417358398438
356
127.80152130126953
357
180.7003631591797
358
125.83745574951172
359
142.02281188964844
360
129.4736785888672
361
142.36341857910156
362
139.03607177734375
363
114.58817291259766
364
129.55850219726562
365
137.2078857421875
366
181.2335662841797
367
164.54595947265625
368
131.04855346679688
0
1

324
146.91802978515625
325
107.8730239868164
326
184.04220581054688
327
151.49977111816406
328
160.87936401367188
329
129.2969970703125
330
144.65005493164062
331
158.44464111328125
332
157.9951171875
333
132.13787841796875
334
170.50314331054688
335
145.41006469726562
336
131.11546325683594
337
113.39952087402344
338
140.61819458007812
339
116.695556640625
340
126.83940887451172
341
143.03819274902344
342
158.6551971435547
343
124.08126831054688
344
144.54360961914062
345
89.59159851074219
346
171.8748016357422
347
112.89038848876953
348
142.80540466308594
349
111.99110412597656
350
148.4212188720703
351
152.26931762695312
352
112.73898315429688
353
144.4576416015625
354
109.77108001708984
355
129.89718627929688
356
122.60802459716797
357
143.64935302734375
358
149.03968811035156
359
161.41790771484375
360
159.98101806640625
361
157.22021484375
362
161.87228393554688
363
160.9307098388672
364
113.12004089355469
365
131.6380615234375
366
130.09066772460938
367
115.12918853759766
368
16